In [24]:
%matplotlib inline
import numpy as np
import os
import time
import h5py
import keras
import pandas as pd
import math
import joblib
import json
import matplotlib.pyplot as plt

from fuel.datasets.hdf5 import H5PYDataset

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from IPython.display import display

from keras.layers import (Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, 
                          Activation, Dropout, Conv2D, Conv2DTranspose,
                          Concatenate, Add, Multiply)
from keras.regularizers import l2
from keras.initializers import RandomUniform
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import Model
from keras import metrics
from keras import backend as K
from keras_tqdm import TQDMNotebookCallback
from keras.datasets import mnist

from resnet import *

## Variational Autoencoder Parameters

In [2]:
img_rows, img_cols, img_chns = 32, 32, 3
original_img_size = (img_rows, img_cols, img_chns)

batch_size = os.environ.get('BATCH_SIZE', 25)
latent_dim = os.environ.get('LATENT_DIM', 256)
intermediate_dim = os.environ.get('INTERMEDIATE_DIM', 1024)
epsilon_std = 1.0
epochs = os.environ.get('EPOCHS', 1000)
activation = os.environ.get('ACTIVATION', 'sigmoid')
dropout = os.environ.get('DROPOUT', 0.0)
decay = os.environ.get('DECAY', 0.0)
learning_rate = os.environ.get('LEARNING_RATE', 0.001)
resnet_depth = os.environ.get('RESNET_DEPTH', 3)

## Load CIFAR10 dataset

In [3]:
ftrain = H5PYDataset("../../data/cifar10/cifar10.hdf5", which_sets=('train',))
X_train, y_train = ftrain.get_data(ftrain.open(), slice(0, ftrain.num_examples))
X_train = np.moveaxis(X_train[:], 1, 3)
X_train = X_train / 255.

ftest = H5PYDataset("../../data/cifar10/cifar10.hdf5", which_sets=('test',))
X_test, y_test = ftest.get_data(ftest.open(), slice(0, ftest.num_examples))
X_test = np.moveaxis(X_test[:], 1, 3)
X_test = X_test / 255.

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


# Helper Functions

In [4]:
#def create_enc_conv_layers(stage, **kwargs):
#    assert K.image_data_format() == 'channels_last'
#    
#    conv_name = '_'.join(['enc_conv', str(stage)])
#    bn_name = '_'.join(['enc_bn', str(stage)])
#    bn_axis = 3
#    layers = [
#        Conv2D(name=conv_name, **kwargs),
#        BatchNormalization(axis=bn_axis, name=bn_name),
#        Activation(activation),
#    ]
#    return layers
#
#def create_dec_trans_conv_layers(stage, **kwargs):
#    assert K.image_data_format() == 'channels_last'
#    conv_name = '_'.join(['dec_trans_conv', str(stage)])
#    bn_name = '_'.join(['dec_bn', str(stage)])
#    bn_axis = 3
#    layers = [
#        Conv2DTranspose(name=conv_name, **kwargs),
#        BatchNormalization(axis=bn_axis, name=bn_name),
#        Activation(activation),
#    ]
#    return layers

def create_dense_layers(stage, width):
    dense_name = '_'.join(['enc_conv', str(stage)])
    bn_name = '_'.join(['enc_bn', str(stage)])
    layers = [
        Dense(width, name=dense_name),
        BatchNormalization(name=bn_name),
        Activation(activation),
        Dropout(dropout),
    ]
    return layers

def inst_layers(layers, in_layer):
    x = in_layer
    for layer in layers:
        if isinstance(layer, list):
            x = inst_layers(layer, x)
        else:
            x = layer(x)
        
    return x

def sampling(args, batch_size=batch_size, latent_dim=latent_dim, epsilon_std=epsilon_std):
    z_mean, z_log_var = args
    
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    
    return z_mean + K.exp(z_log_var) * epsilon


def resnet_layers(x, layers, depth, stage_base, transpose=False):
    assert depth in [0, 1, 2, 3]
    
    filters = [64, 64, 256]
    x, layer = conv_block(x, 3, filters, stage=stage_base + 2, block='a', strides=(1, 1), transpose=transpose)
    layers.add(layer)
    if depth >= 2:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 2, block='b')
        layers.add(layer)
    if depth >= 3:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 2, block='c')
        layers.add(layer)
   
    filters = [128, 128, 512]
    x, layer = conv_block(x, 3, filters, stage=stage_base + 3, block='a', transpose=transpose)
    layers.add(layer)
    if depth >= 1:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 3, block='b')
        layers.add(layer)
    if depth >= 2:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 3, block='c')
        layers.add(layer)
    if depth >= 3:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 3, block='d')
        layers.add(layer)
    
    filters = [256, 256, 1024]
    x, layer = conv_block(x, 3, filters, stage=stage_base + 4, block='a', transpose=transpose)
    layers.add(layer)
    if depth >= 1:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 4, block='b')
        layers.add(layer)
    if depth >= 2:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 4, block='c')
        layers.add(layer)
        x, layer = identity_block(x, 3, filters, stage=stage_base + 4, block='d')
    if depth >= 3:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 4, block='e')
        layers.add(layer)
        x, layer = identity_block(x, 3, filters, stage=stage_base + 4, block='f')
        layers.add(layer)
   
    filters =  [512, 512, 2048]
    x, layer = conv_block(x, 3, filters, stage=stage_base + 5, block='a', transpose=transpose)
    layers.add(layer)
    if depth >= 2:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 5, block='b')
        layers.add(layer)
    if depth >= 3:
        x, layer = identity_block(x, 3, filters, stage=stage_base + 5, block='c')
        layers.add(layer)
    
    return x, layers

## Loss Function

In [13]:
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1. + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
   
    return K.mean(kl_loss)

def logx_loss(x, x_decoded_mean):
    x = K.flatten(x)
    x_decoded_mean = K.flatten(x_decoded_mean)
    xent_loss = img_rows * img_cols * img_chns * metrics.binary_crossentropy(x, x_decoded_mean)
    return xent_loss

def vae_loss(x, x_decoded_mean):
    return logx_loss(x, x_decoded_mean) + kl_loss(x, x_decoded_mean)

# VAE

In [6]:
layers = []
x_input = Input(batch_shape=(batch_size,) + original_img_size)
x, layers = resnet_layers(x_input, layers, depth=resnet_depth, stage_base=0)

encoder_layers = [
    create_dense_layers(stage=9, width=intermediate_dim),
    Flatten(),
]
_enc_dense = inst_layers(encoder_layers, x)

_z_mean_1 = Dense(latent_dim, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(_enc_dense)
_z_log_var_1 = Dense(latent_dim, kernel_regularizer=l2(0.1), bias_regularizer=l2(0.1))(_enc_dense)
z_mean = _z_mean_1
z_log_var = _z_log_var_1

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

decoder_layers = [
    create_dense_layers(stage=10, width=intermediate_dim),
    Reshape((4, 4, intermediate_dim // 16)),
]
_dec_out = inst_layers(decoder_layers, z)
_dec_out, decoder_layers = resnet_layers(_dec_out, decoder_layers, depth=resnet_depth,
                                         transpose=True, stage_base=10)

out_layer = Conv2DTranspose(name='x_decoded', filters=3, kernel_size=1, strides=1, activation='sigmoid')
decoder_layers.append(out_layer)
_output = out_layer(_dec_out)

# Encoder

In [7]:
vae = Model(inputs=x_input, outputs=_output)
optimizer = Adam(lr=learning_rate, decay=decay)
vae.compile(optimizer=optimizer, loss=vae_loss)
vae.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (25, 32, 32, 3)       0                                            
____________________________________________________________________________________________________
res2a_branch2a (Conv2D)          (25, 32, 32, 64)      256         input_1[0][0]                    
____________________________________________________________________________________________________
bn2a_branch2a (BatchNormalizatio (25, 32, 32, 64)      256         res2a_branch2a[0][0]             
____________________________________________________________________________________________________
activation_1 (Activation)        (25, 32, 32, 64)      0           bn2a_branch2a[0][0]              
___________________________________________________________________________________________

In [8]:
start = time.time()

early_stopping = keras.callbacks.EarlyStopping('val_loss', min_delta=0.1, patience=50)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=25, min_lr=0.001 * learning_rate)
callbacks=[early_stopping, reduce_lr]
if 'CMDLINE' not in os.environ:
    callbacks += [TQDMNotebookCallback()]

history = vae.fit(
    X_train, X_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(X_test, X_test),
    verbose=0
)

done = time.time()
elapsed = done - start
print("Elapsed: ", elapsed)

49975/|/[loss: 1903.034] 100%|| 49975/50000 [30:47<00:00, 27.31it/s]]s]]

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(history.history)
display(df.describe(percentiles=[0.25 * i for i in range(4)] + [0.95, 0.99]))
df.plot(figsize=(8, 6))

In [14]:
# Eval kl loss
m = Model(inputs=x_input, outputs=_output)
optimizer = Adam(lr=learning_rate, decay=decay)
m.compile(optimizer=optimizer, loss=kl_loss)
val_kl_loss = m.evaluate(x=X_test, y=X_test, batch_size=batch_size)

# Eval logx loss
m = Model(inputs=x_input, outputs=_output)
optimizer = Adam(lr=learning_rate, decay=decay)
m.compile(optimizer=optimizer, loss=logx_loss)
val_logx_loss = m.evaluate(x=X_test, y=X_test, batch_size=batch_size)

print()
print("kl_loss = %.2f" % val_kl_loss)
print("logx_loss = %.2f" % val_logx_loss)

10000/10000 [==============================] - 134s     

In [15]:
g_z = Input(shape=(latent_dim,))
g_output = inst_layers(decoder_layers, g_z)
generator = Model(g_z, g_output)

ValueError: Input 0 is incompatible with layer x_decoded: expected axis -1 of input shape to have value 2048 but got shape (None, 4, 4, 64)

In [ ]:
import matplotlib.pyplot as plt
n = 10
figure = np.zeros((img_rows * n, img_cols * n, img_chns))

for j in range(n):
    for i in range(n):
        z_sample = np.random.normal(size=latent_dim).reshape(1, latent_dim)
        x_decoded = generator.predict(z_sample, batch_size=1)
        digit = x_decoded.reshape(img_rows, img_cols, img_chns)
        
        d_x = i * img_rows
        d_y = j * img_cols
        figure[d_x:d_x + img_rows, d_y:d_y + img_cols] = digit
        
plt.figure(figsize=(10, 10))
plt.imshow(figure)
plt.show()

In [36]:
if os.environ.get('OUTDIR', None):
    generator.save(os.path.join(os.environ['OUTDIR'], 'generator-depth-' + str(resnet_depth) + '.h5'))
    vae.save(os.path.join(os.environ['OUTDIR'], 'vae-depth-' + str(resnet_depth) + '.h5'))

    vals = {k: v for k, v in locals().items() if type(v) in [int, float, bool]}
    with open(os.path.join(os.environ['OUTDIR'], 'params.json'), 'w') as f:
        json.dump(vals, f)

NameError: name 'generator' is not defined